<a href="https://colab.research.google.com/github/kimwoonggon/kostat_giro_economics/blob/master/(%EA%B1%B4%EC%84%A4%EA%B2%BD%EA%B8%B0%EB%8F%99%ED%96%A5%EC%A1%B0%EC%82%AC)%EA%B2%BD%EC%9D%B8%EC%B2%AD_%EA%B1%B4%EC%84%A4_%EB%AF%BC%EA%B0%84%EC%9E%90%EB%A3%8C_%EC%88%98%EC%A7%91%EA%B4%80%EB%A6%AC%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
## 이 버전은 담당자별로 자료를 수집하여 담당자가 조사하는 건설사 별로 자료를 분류
## 최종 수정 날짜 : '20. 2. 19

import os
import urllib.request
import sys
import datetime
import time
import json
import pandas as pd
import re
import collections
import glob
import numpy as np
from multiprocessing import Pool, freeze_support
from functools import partial
from itertools import repeat

path = "/content/gdrive/My Drive/Colab Notebooks/건설경기동향조사"
class gunseol_crawl:
    # api_id, api_password에 네이버 개발자 센터에서 발급받은 api_id , api_password를 입력해 주셔야 사용 가능합니다.

    def __init__(self, file_path, api_id='', api_password='', multiprocessing=False):
        self.total_data = pd.DataFrame(columns=['날짜', '제목', '기사요약', '링크', '담당자', '사업체명'])
        self.file_path = file_path
        self.api_id = api_id
        self.api_password = api_password
        
        self.user_search_lists = self.load_gunseol_list()
        if multiprocessing is False:
            for self.user_search_list in self.user_search_lists:
                
                print(self.user_search_list[0]+'님의 건설경기 자료를 수집하겠습니다\n')

                print(self.user_search_list)
                self.crawl_news_data(self.user_search_list[1:], self.user_search_list[0])
                print('%s님 수고하셨습니다.' % self.user_search_list[0])
            self.save_to_excel()


        else:
            with Pool(3) as pool:
                for user_search_list in user_search_lists:
                    func = partial(self.crawl_news_data, user_name = user_search_list[0])
                    pool.map(func, user_search_list[1:])

    def load_gunseol_list(self):
        gunseol_list = pd.read_csv(self.file_path, encoding='cp949', engine='python')
        total_gunseol_list = []
        name_list = gunseol_list['담당자'].unique().tolist()
        for name in name_list:
            gun = gunseol_list.loc[gunseol_list['담당자'] == name_list[name_list.index(name)]]['기업체명'].tolist()
            re_gun = []
            for i in gun:
                del_letter = re.compile('\((주)\)')
                i = del_letter.sub("", i)

                re_gun.append(i)
            re_gun.insert(0, name)
            total_gunseol_list.append(re_gun)

        return total_gunseol_list

    def get_naver_connection(self, url):

        req = urllib.request.Request(url)
        req.add_header('X-Naver-Client-Id', self.api_id)
        req.add_header('X-Naver-Client-Secret', self.api_password)


        try:
            response = urllib.request.urlopen(req)

            if response.getcode() == 200:
                return response.read().decode('utf-8')

        except Exception as e:
            return None



    def get_naver_json(self, kind, search_text, page_start, display_num):
        base = 'https://openapi.naver.com/v1/search'
        node = '/%s.json' % kind
        parameters = '?query=%s&start=%s&display=%s' % (urllib.parse.quote(search_text), page_start, display_num)
        url = base + node + parameters


        raw_json = self.get_naver_connection(url)

        if raw_json == None:
            return None
        else:
            return json.loads(raw_json)

    def process_data(self, post, jsonResult):


        pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
        # 수집 기간은 28일
        if (datetime.datetime.now() - pDate).total_seconds() > int(86400 * 7 * 4):
            return None
        else:
            pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
            title = post['title']

        del_letters1 = re.compile('[<b>|</b>|]')
        del_letters2 = re.compile('[&quot]')

        title = del_letters1.sub("",title)
        title = del_letters2.sub("",title)
        description = post['description']
        description = del_letters1.sub("",description)
        description = del_letters2.sub("",description)

        link = post['originallink']

        description, title = self.filter_article(description=description, title=title)
        if description is not None and title is not None:

            sys.stdout.write(title+"\n")
            jsonResult.append({'제목':title, '기사요약':description, '링크':link, '날짜':pDate})

        return None

    def filter_article(self, description, title):
        # 아시아, 중동
        country_list = ['일본', '중국', '러시아', '대만', '필리핀', '브루나이', '인도네시아', '동티모르', '파푸아뉴기니',
                        '싱가포르', '말레이시아', '캄보디아', '베트남', '라오스', '태국', '미얀마', '방글라데시', '부탄', '네팔',
                        '스리랑카', '인도', '몰디브', '파키스탄', '카자흐스탄', '키르키스스탄', '우즈베키스탄', '투르크메니스탄',
                        '아프가니스탄', '이란', '이라크', '쿠웨이트', '바레인', '아랍에미리트', 'UAE', '시리아', '터키', '레바논',
                        '요르단', '이스라엘', '사우디아라비아', '카타르', '오만', '예멘', '이집트', '알제리', '아시안','중동', '동남아', '사우디']

        country_list_2 = ['체코', '폴란드', '유럽', '스페인', '이탈리아']

        country_list_3 = ['멕시코', '남미', '아프리카', '에티오피아', '나이지리아','파나마', '모잠비크']

        filter_s = country_list + country_list_2 + country_list_3
        for filter in filter_s:
            if filter in description or filter in title:

                return None, None

        return description, title

    def crawl_news_data(self, user_search_list, user_name):

        display_count = 100
        kind = 'news'

        for search in user_search_list:
            self.jsonResult = []
            pre_search_text = '"수주"'
            real_search_text = search
            search_text = real_search_text + ' ' + pre_search_text
            jsonSearch = self.get_naver_json(kind, search_text, 1, display_count)

            while ((jsonSearch != None) and (jsonSearch['display'] != 0)):

                for post in jsonSearch['items']:
                    self.process_data(post, self.jsonResult)

                nStart = jsonSearch['start'] + jsonSearch['display']
                jsonSearch = self.get_naver_json(kind, search_text, nStart, display_count)

            with open("%s.json" % search, 'w', encoding='utf8') as f:
                result_json = json.dumps(self.jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
                f.write(result_json)

            data = pd.read_json("%s.json" % search, encoding='utf-8')
            data = pd.DataFrame(data, columns = ['날짜', '제목', '기사요약', '링크'])

            data['담당자'] = user_name
            data['사업체명'] = real_search_text

            self.total_data = pd.concat([self.total_data, data], axis=0)
            del self.jsonResult

        jsonFileList = glob.glob('*.json')
        for jsonFile in jsonFileList:
            os.remove(jsonFile)
        print('-' * 50)
        print('수집 완료!')
        print('-' * 50)

    def save_to_excel(self):
        filename = os.path.join(path,"경제조사과_건설수집자료" + datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S") + '.xlsx')
        self.total_data.reset_index(drop=True)
        self.total_data = self.total_data[['담당자','사업체명','날짜', '제목', '기사요약', '링크']]
        self.total_data.to_excel(filename, index=False)

if __name__ == '__main__':
    c = time.time()
    crawl = gunseol_crawl("/content/gdrive/My Drive/Colab Notebooks/건설경기동향조사/건설명부(202010).csv", multiprocessing=False)
    d = time.time()
    print("crawling time is : %d 초" % (d-c))

In [0]:
!pip install keras-bert
!pip install keras_radam
import tensorflow as tf
import pandas as pd
import os
import numpy as np
from keras.models import load_model
from keras import backend as K
from keras.layers import Add, Lambda, Dense, Embedding, Bidirectional, LSTM, Concatenate, BatchNormalization, Dropout, GlobalMaxPooling1D
from keras.layers import Reshape, Activation
from keras import Input, Model
from keras import optimizers
from keras.callbacks import ModelCheckpoint


%tensorflow_version 1.x
import shutil
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
from tqdm import tqdm

import codecs
from keras_bert import Tokenizer
import re
import pickle
from keras_bert import get_pretrained, PretrainedList, get_checkpoint_paths
from keras_radam import RAdam
import keras as keras
import keras.backend as K

path = "gdrive/My Drive/Colab Notebooks/건설경기동향조사"
filename = "경제조사과_건설수집자료2020-02-17-154045.xlsx"

test_naver = pd.read_excel(os.path.join(path, filename))

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
        if not self._cased:
            
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

def predict_convert_data(data_df):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def predict_load_data(x):
    data_df = x 
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

!pip install keras-bert

try:
  os.makedirs("etri_bert")
except:
  pass

try:
  copytree(os.path.join(path,"etri_bert"), "etri_bert")
except:
  pass

SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS=2
LR=1e-5

pretrained_path ="etri_bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'etri_vocab.txt')

DATA_COLUMN = "제목"
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
          token = token.replace("_","")
          token = "##" + token
        token_dict[token] = len(token_dict)

tokenizer = inherit_Tokenizer(token_dict)



layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)


def get_gunseol_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output

  outputs = keras.layers.Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)



  semi_bert_model = keras.models.Model(inputs, outputs)
  semi_bert_model.compile(
      optimizer=RAdam(learning_rate=0.00001, weight_decay=0.0025),
      loss='binary_crossentropy',
      metrics=['accuracy']
  )
  
  return semi_bert_model


bert_model = get_gunseol_model(model)

In [0]:
import datetime
bert_model.load_weights(os.path.join(path, "semi_bert_model.h5"))
test_naver['제목'] = test_naver['제목'].astype(str) + test_naver['기사요약'].astype(str)
test_naver_x = predict_load_data(test_naver)

naver_predict = bert_model.predict(test_naver_x)


100%|██████████| 5603/5603 [00:02<00:00, 2660.44it/s]


In [0]:
test_naver['기사여부'] = np.round(np.squeeze(naver_predict),0)
test_naver.head(10)
test_naver.to_excel(os.path.join(path, datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S") + "_필터처리분.xlsx"), index=False)